[Cutom Training Loop:](https://www.scaler.com/topics/keras/customizing-training-loops-keras/)

In [2]:
import matplotlib.pyplot
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

layers = keras.layers
regularizers = keras.regularizers
mnist = keras.datasets.mnist

#HYPERPARAMETERS
BATCH_SIZE = 32
WEIGHT_DECAY = 0.001
LEARNING_RATE = 0.001
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

In [4]:
model = keras.Sequential(
    [
        layers.Input(shape=(28, 28, 1)),
        layers.Conv2D(64, (3, 3), padding="same"),
        layers.ReLU(),
        layers.Conv2D(128, (3, 3), padding="same"),
        layers.ReLU(),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="model",
)

In [27]:
class CustomModel(keras.Model):
    def __init__(self, model):
        super(CustomModel, self).__init__()
        self.model = model

    def compile(self, optimizer, loss):
        super(CustomModel, self).compile()
        self.optimizer = optimizer
        self.loss = loss

    def train_step(self, data):
        x,y = data

        with tf.GradientTape() as tape:
            y_pred = self.model(x, training=True) # forward pass
            loss = self.loss(y, y_pred)
        # compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        #update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        #update metrics
        acc_metric.update_state(y, y_pred)
        # for metric in self.metrics:
        #     if(metric.name== "loss"):
        #         metric.update_state(loss)
        #     else:
        #         metric.update_state(y, y_pred)
        return {"loss": loss, "accurecy": acc_metric.result()}
    
    def test_step(self, data):
        x,y = data
        y_pred = self.model(x, training= False)
        loss = self.loss(y, y_pred)
        acc_metric.update_state(y, y_pred)

        return {"loss": loss, "accurecy": acc_metric.result()}


In [28]:
acc_metric = keras.metrics.SparseCategoricalAccuracy(name="accuracy")
training = CustomModel(model)
training.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

training.fit(x_train, y_train, batch_size=64, epochs=2)

Epoch 1/2
938/938 [==============================] - 235s 246ms/step - loss: 0.0103 - accurecy: 0.9974
Epoch 2/2
938/938 [==============================] - 245s 261ms/step - loss: 0.0053 - accurecy: 0.9976


In [29]:
training.evaluate(x_test, y_test, batch_size=32)

313/313 [==============================] - 11s 33ms/step - loss: 0.0541 - accurecy: 0.9973


[0.9969461560249329, 1.341100869467482e-06]